In [1]:
import requests
import json

url = "http://openapi.seoul.go.kr:8088/4954576d487768643638684a565a75/json/SearchSTNBySubwayLineInfo/1/768/"
response = requests.get(url)
data = json.loads(response.text)
print(json.dumps(data, indent=4,ensure_ascii=False))

# 데이터 가공 및 MySQL에 삽입하는 코드 작성

{
    "SearchSTNBySubwayLineInfo": {
        "list_total_count": 768,
        "RESULT": {
            "CODE": "INFO-000",
            "MESSAGE": "정상 처리되었습니다"
        },
        "row": [
            {
                "STATION_CD": "0244",
                "STATION_NM": "용답",
                "STATION_NM_ENG": "Yongdap",
                "LINE_NUM": "02호선",
                "FR_CODE": "211-1",
                "STATION_NM_CHN": "龍踏",
                "STATION_NM_JPN": "龍踏"
            },
            {
                "STATION_CD": "0245",
                "STATION_NM": "신답",
                "STATION_NM_ENG": "Sindap",
                "LINE_NUM": "02호선",
                "FR_CODE": "211-2",
                "STATION_NM_CHN": "新踏",
                "STATION_NM_JPN": "新踏"
            },
            {
                "STATION_CD": "0250",
                "STATION_NM": "용두",
                "STATION_NM_ENG": "Yongdu",
                "LINE_NUM": "02호선",
                "FR_CODE": "211-3",
             

In [2]:
# 필요한 데이터 추출
extracted_data = []
for row in data["SearchSTNBySubwayLineInfo"]["row"]:
    extracted_data.append({
        "STATION_CD": row["STATION_CD"],
        "STATION_NM": row["STATION_NM"],
        "LINE_NUM": row["LINE_NUM"]
    })

# 추출된 데이터 출력
print(json.dumps(extracted_data, indent=4, ensure_ascii=False))




[
    {
        "STATION_CD": "0244",
        "STATION_NM": "용답",
        "LINE_NUM": "02호선"
    },
    {
        "STATION_CD": "0245",
        "STATION_NM": "신답",
        "LINE_NUM": "02호선"
    },
    {
        "STATION_CD": "0250",
        "STATION_NM": "용두",
        "LINE_NUM": "02호선"
    },
    {
        "STATION_CD": "0336",
        "STATION_NM": "학여울",
        "LINE_NUM": "03호선"
    },
    {
        "STATION_CD": "0428",
        "STATION_NM": "삼각지",
        "LINE_NUM": "04호선"
    },
    {
        "STATION_CD": "0429",
        "STATION_NM": "신용산",
        "LINE_NUM": "04호선"
    },
    {
        "STATION_CD": "1008",
        "STATION_NM": "이촌",
        "LINE_NUM": "경의선"
    },
    {
        "STATION_CD": "1014",
        "STATION_NM": "청량리",
        "LINE_NUM": "경의선"
    },
    {
        "STATION_CD": "1286",
        "STATION_NM": "운천",
        "LINE_NUM": "경의선"
    },
    {
        "STATION_CD": "1407",
        "STATION_NM": "온양온천",
        "LINE_NUM": "01호선"
    },
    {
        "

In [3]:
import MySQLdb
import json

# MySQL 연결 설정
connection = MySQLdb.connect(user="Chuldong", passwd="Chuldong123!", host="localhost", db="Capital_Subway_DB", port=3307)
cursor = connection.cursor()



# 데이터 삽입
for row in extracted_data:
    station_cd = row["STATION_CD"]
    station_nm = row["STATION_NM"]
    line_num = row["LINE_NUM"]
    
    # 쿼리 실행
    cursor.execute("INSERT INTO subway_info (STATION_CD, STATION_NM, LINE_NUM) VALUES (%s, %s, %s)", (station_cd, station_nm, line_num))

# 변경사항 커밋
connection.commit()

# 연결 종료
connection.close()


#### MySQL → Python으로 다시 가져오기

In [19]:
import MySQLdb
import pandas as pd

# MySQL connection setup
connection = MySQLdb.connect(user="Chuldong", passwd="Chuldong123!", host="localhost", db="Capital_Subway_DB", port=3307)

# Create a cursor
cursor = connection.cursor()

# Retrieve data
cursor.execute("SELECT * FROM subway_info")

# Fetch all rows
rows = cursor.fetchall()

# Create a DataFrame from the fetched rows
Station_Info = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close the cursor and connection
cursor.close()
connection.close()

# Display the DataFrame
Station_Info

,STATION_CD,STATION_NM,LINE_NUM
0,0150,서울역,01호선
1,0151,시청,01호선
2,0152,종각,01호선
3,0153,종로3가,01호선
4,0154,종로5가,01호선
...,...,...,...
763,4925,걸포북변,김포도시철도
764,4926,사우,김포도시철도
765,4927,풍무,김포도시철도
766,4928,고촌,김포도시철도


In [39]:
# 문자열 앞의 0 제거
Station_Info['LINE_NUM'] = Station_Info['LINE_NUM'].str.lstrip('0')
Station_Info


,STATION_CD,STATION_NM,LINE_NUM,Facility
0,0150,서울역,1호선,
1,0151,시청,1호선,
2,0152,종각,1호선,
3,0153,종로3가,1호선,
4,0154,종로5가,1호선,
...,...,...,...,...
763,4925,걸포북변,김포도시철도,
764,4926,사우,김포도시철도,
765,4927,풍무,김포도시철도,
766,4928,고촌,김포도시철도,


In [44]:
Station_Info['LINE_NUM'].unique()

array(['1호선', '2호선', '경의선', '3호선', '4호선', '수인분당선', '경춘선', '경강선', '5호선',
       '6호선', '7호선', '8호선', '인천선', '인천2호선', '9호선', '공항철도', '신분당선', '신림선',
       '용인경전철', '의정부경전철', '우이신설경전철', '서해선', '김포도시철도'], dtype=object)

10 : 경의선<br>
11 : 경춘선<br>
12 : 경강선<br>
13 : 수인분당<br>
14 : 신분당<br>
15 : 에버<br>
16 : 의정부경전철<br>
17 : 인천선<br>
18 : 인천2호선<br>
19 : 공항철도<br>
20 : 신림선<br>
21 : 용인경전철<br>
22 : 우이신설경전철<br>
23 : 서해선<br>
24 : 김포도시철도

In [46]:
# LINE_NUM 매핑 딕셔너리
line_mapping = {
    '1호선': '1',
    '2호선': '2',
    '3호선': '3',
    '4호선': '4',
    '5호선': '5',
    '6호선': '6',
    '7호선': '7',
    '8호선': '8',
    '9호선': '9',
    # 나머지 매핑값 추가
    '10':'경의선',
    '11':'경춘선',
    '12':'경강선',
    '13':'수인분당',
    '14':'신분당',
    '15':'에버',
    '16':'의정부경전철',
    '17':'인천선',
    '18':'인천2호선',
    '19':'공항철도',
    '20':'신림선',
    '21':'용인경전철',
    '22':'우이신설경전철',
    '23':'서해선',
    '24':'김포도시철도'
}

# 'LINE' 컬럼 추가
Station_Info['LINE'] = Station_Info['LINE_NUM'].map(line_mapping).fillna('')

Station_Info.head(30)

,STATION_CD,STATION_NM,LINE_NUM,Facility,LINE
0,0150,서울역,1호선,,1
1,0151,시청,1호선,,1
2,0152,종각,1호선,,1
3,0153,종로3가,1호선,,1
4,0154,종로5가,1호선,,1
5,0155,동대문,1호선,,1
6,0156,신설동,1호선,,1
7,0157,제기동,1호선,,1
8,0158,청량리,1호선,,1
9,0159,동묘앞,1호선,,1


In [47]:
Station_Info['Facility'] = ''
Station_Info

,STATION_CD,STATION_NM,LINE_NUM,Facility,LINE
0,0150,서울역,1호선,,1
1,0151,시청,1호선,,1
2,0152,종각,1호선,,1
3,0153,종로3가,1호선,,1
4,0154,종로5가,1호선,,1
...,...,...,...,...,...
763,4925,걸포북변,김포도시철도,,
764,4926,사우,김포도시철도,,
765,4927,풍무,김포도시철도,,
766,4928,고촌,김포도시철도,,


In [50]:
dict = {'EV':{},'ES':{},\
        'Wheel_Chair_Lift':{},'Wheel_Chair_Ramp':{},'Feeding_Room':{}}
dict.keys()

dict_keys(['EV', 'ES', 'Wheel_Chair_Lift', 'Wheel_Chair_Ramp', 'Feeding_Room'])

### 엘리베이터 시작

In [10]:
# 이 셀 실행은 일단 보류.
EV_Line_1_Df =  pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권1호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_2_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권2호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_3_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권3호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_4_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권4호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_5_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권5호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_6_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권6호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_7_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권7호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_8_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수도권8호선_엘리베이터_20221122.csv', encoding='cp949')
EV_Line_Gyeongui_Jungang_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_경의중앙선_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Gyeongchun_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_경춘선_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Suin_Bundang_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_수인분당선_승강장_정보_20221121.csv', encoding='cp949')
EV_Line_New_Bundang_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_신분당선_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Ever_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_에버라인_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Uijeongbu_Rail_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_의정부경전철_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Incheon1_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_인천1호선_승강장_정보_20221013.csv', encoding='cp949')
EV_Line_Incheon2_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/국가철도공단_인천2호선_승강장_정보_20221013.csv', encoding='cp949')


In [81]:
EV_1to8_Line_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/서울시 지하철 역사 노약자 장애인 편의시설 현황/(최종 전처리) 서울교통공사_엘리베이터 설치현황_20230420.csv',encoding='cp949')
EV_1to8_Line_Df.head(20)

,연번,호선,역명,호기,승강기번호,설치위치,운행구간
0,0,1,서울역,내부#1,0074-205,"시청 방면5-1, B2-B1",B2-B1
1,1,1,서울역,외부#1,0104-656,"2번 출구 측, B1-1F",B1-1F
2,2,1,서울역,외부#2,0105-867,"4번 출구 측, B1-1F",B1-1F
3,3,1,서울역,외부#3,0073-979,"3번 출구 측, B1-1F",B1-1F
4,4,1,시청,내부#1,0002-888,"종각 방면3-2, B2-B1",B2-B1
5,5,1,시청,내부#2,0002-887,"서울역 방면8-3, B2-B1",B2-B1
6,6,1,시청,외부#1,0002-889,"1,2번 출구 사이, B1-1F",B1-1F
7,7,1,종각,내부#1,0000-506,"종로3가 방면6-4, B2-B1",B2-B1
8,8,1,종각,내부#2,0000-507,"시청 방면4-4, B2-B1",B2-B1
9,9,1,종각,외부#1,0000-508,"3번 출구 측, B1-1F",B1-1F


In [82]:
EV_1to8_Line_Df = EV_1to8_Line_Df.merge(Station_Info[['STATION_NM', 'LINE', 'STATION_CD']], left_on=['역명', '호선'], right_on=['STATION_NM', 'LINE'], how='left')
EV_1to8_Line_Df

,연번,호선,역명,호기,승강기번호,설치위치,운행구간,STATION_NM,LINE,STATION_CD
0,0,1,서울역,내부#1,0074-205,"시청 방면5-1, B2-B1",B2-B1,서울역,1,0150
1,1,1,서울역,외부#1,0104-656,"2번 출구 측, B1-1F",B1-1F,서울역,1,0150
2,2,1,서울역,외부#2,0105-867,"4번 출구 측, B1-1F",B1-1F,서울역,1,0150
3,3,1,서울역,외부#3,0073-979,"3번 출구 측, B1-1F",B1-1F,서울역,1,0150
4,4,1,시청,내부#1,0002-888,"종각 방면3-2, B2-B1",B2-B1,시청,1,0151
...,...,...,...,...,...,...,...,...,...,...
825,825,8,수진,2,2116-590,"모란 방면4-1, B2-B1",B2-B1,수진,8,2826
826,826,8,수진,3,2212-625,"3번 출구 측, B1-1F",B1-1F,수진,8,2826
827,827,8,모란,1,2116-583,"수진 방면5-2, B3-B2-B1",B3-B2-B1,모란,8,2827
828,828,8,모란,2,2116-584,"모란 방면2-3, B3-B2",B3-B2,모란,8,2827


In [83]:
# 삭제할 컬럼 인덱스 리스트
column_indices_to_drop = [0,7,8]

# 여러 컬럼 삭제
EV_1to8_Line_Df = EV_1to8_Line_Df.drop(EV_1to8_Line_Df.columns[column_indices_to_drop], axis=1)
EV_1to8_Line_Df

,호선,역명,호기,승강기번호,설치위치,운행구간,STATION_CD
0,1,서울역,내부#1,0074-205,"시청 방면5-1, B2-B1",B2-B1,0150
1,1,서울역,외부#1,0104-656,"2번 출구 측, B1-1F",B1-1F,0150
2,1,서울역,외부#2,0105-867,"4번 출구 측, B1-1F",B1-1F,0150
3,1,서울역,외부#3,0073-979,"3번 출구 측, B1-1F",B1-1F,0150
4,1,시청,내부#1,0002-888,"종각 방면3-2, B2-B1",B2-B1,0151
...,...,...,...,...,...,...,...
825,8,수진,2,2116-590,"모란 방면4-1, B2-B1",B2-B1,2826
826,8,수진,3,2212-625,"3번 출구 측, B1-1F",B1-1F,2826
827,8,모란,1,2116-583,"수진 방면5-2, B3-B2-B1",B3-B2-B1,2827
828,8,모란,2,2116-584,"모란 방면2-3, B3-B2",B3-B2,2827


In [87]:
# 원하는 컬럼 순서로 컬럼을 재배치하고 새로운 데이터프레임 생성
new_columns = ['STATION_CD', '호선', '역명','승강기번호','호기','설치위치','운행구간']
EV_1to8_Line_DF = EV_1to8_Line_Df[new_columns]

# 값과 컬럼 순서를 함께 변경
EV_1to8_Line_DF = EV_1to8_Line_DF.reindex(columns=new_columns)
EV_1to8_Line_DF
#EV_1to8_Line_DF[EV_1to8_Line_DF['역명']=="시청"]

,STATION_CD,호선,역명,승강기번호,호기,설치위치,운행구간
4,0151,1,시청,0002-888,내부#1,"종각 방면3-2, B2-B1",B2-B1
5,0151,1,시청,0002-887,내부#2,"서울역 방면8-3, B2-B1",B2-B1
6,0151,1,시청,0002-889,외부#1,"1,2번 출구 사이, B1-1F",B1-1F
37,0201,2,시청,0042-520,내부#1,"충정로 방면6-2, 을지로입구 방면5-3, B3-B1",B3-B1
38,0201,2,시청,0002-886,외부#1,"9번 출구 측, B1-1F",B1-1F


In [91]:
# Station_Facility 딕셔너리 생성
Station_Facility = {}

for idx, row in EV_1to8_Line_Df.iterrows():
    station_cd = str(row['STATION_CD']).strip()
    if station_cd not in Station_Facility:
        Station_Facility[station_cd] = {
            '역명': row['역명'],
            '호선': row['호선'],
            '승강기번호': [row['승강기번호']],
            '설치위치': [row['설치위치']]
        }
    else:
        Station_Facility[station_cd]['승강기번호'].append(row['승강기번호'])
        Station_Facility[station_cd]['설치위치'].append(row['설치위치'])

dict['EV'] = Station_Facility
dict['EV']

{'0150': {'역명': '서울역',
  '호선': '1',
  '승강기번호': ['0074-205', '0104-656', '0105-867', '0073-979'],
  '설치위치': ['시청 방면5-1, B2-B1',
   '2번 출구 측, B1-1F',
   '4번 출구 측, B1-1F',
   '3번 출구 측, B1-1F']},
 '0151': {'역명': '시청',
  '호선': '1',
  '승강기번호': ['0002-888', '0002-887', '0002-889'],
  '설치위치': ['종각 방면3-2, B2-B1', '서울역 방면8-3, B2-B1', '1,2번 출구 사이, B1-1F']},
 '0152': {'역명': '종각',
  '호선': '1',
  '승강기번호': ['0000-506', '0000-507', '0000-508', '0113-945'],
  '설치위치': ['종로3가 방면6-4, B2-B1',
   '시청 방면4-4, B2-B1',
   '3번 출구 측, B1-1F',
   '1번 출구 측, B1-1F']},
 'nan': {'역명': '종로가',
  '호선': '1',
  '승강기번호': ['0000-512',
   '0000-513',
   '0073-426',
   '0000-533',
   '0000-534',
   '0155-440',
   '0042-132',
   '0042-133',
   '0042-130',
   '0042-131',
   '0102-390',
   '0125-744',
   '0125-743',
   '0040-427',
   '0040-428',
   '0114-148',
   '0113-923',
   '0140-721',
   '0140-722',
   '0140-723',
   '0140-724',
   '0140-807',
   '0157-507',
   '0140-808',
   '0140-809',
   '2231-268',
   '2231-269',
   '2231

#### 엘베완료

### 에스컬레이터 시작

In [58]:
ES_1to8_Line_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/서울시 지하철 역사 노약자 장애인 편의시설 현황/(최종 전처리) 서울교통공사_에스컬레이터 설치현황_20230420.csv',encoding='cp949')
ES_1to8_Line_Df.head(20)

,호선,역명,호기,승강기번호,설치위치,운행구간,운행방향
0,1,서울역,1,1800-448,"연결통로(4호선), B2-B1",B2-B1,상행
1,1,서울역,2,1806-600,"남영 방면2-3, B1-B2",B1-B2,하행
2,1,서울역,3,1806-599,"시청 방면9-3, B2-B1",B2-B1,상행
3,1,서울역,4,1810-816,"4번 출구 측, B1-1F",B1-1F,상행
4,1,서울역,5,1810-817,"4번 출구 측, 1F-B1",1F-B1,하행
5,1,시청,1,1808-119,"5번 출구 측, 1F-B1",1F-B1,하행
6,1,시청,2,1808-120,"5번 출구 측, B1-1F",B1-1F,상행
7,1,시청,3,1809-239,"1번 출구 측, B1-1F",B1-1F,상행
8,1,종각,1,1810-730,"1번 출구 측, 1F-B1",1F-B1,하행
9,1,종각,2,1810-729,"1번 출구 측, B1-1F",B1-1F,상행


In [59]:
# Station_Facility 변수 초기화
Station_Facility = {}

# 데이터프레임 정보를 가공하여 Station_Facility 변수에 저장
for _, row in ES_1to8_Line_Df.iterrows():
    station = row['역명']
    if station not in Station_Facility:
        Station_Facility[station] = {
            '호선': row['호선'],
            '승강기번호': [],
            '설치위치': [],
            '운행방향': []
        }
    Station_Facility[station]['승강기번호'].append(row['승강기번호'])
    Station_Facility[station]['설치위치'].append(row['설치위치'])
    Station_Facility[station]['운행방향'].append(row['운행방향'])
    

    
dict['ES'] = Station_Facility
dict['ES']

{'서울역': {'호선': '1',
  '승강기번호': ['1800-448',
   '1806-600',
   '1806-599',
   '1810-816',
   '1810-817',
   '1801-085',
   '1801-373',
   '1801-348',
   '1801-316'],
  '설치위치': ['연결통로(4호선), B2-B1',
   '남영 방면2-3, B1-B2',
   '시청 방면9-3, B2-B1',
   '4번 출구 측, B1-1F',
   '4번 출구 측, 1F-B1',
   '삼각지 방면10-4, 1F-B1',
   '회현 방면1-1, B1-1F',
   '9-1번 출구(교통섬) 측 , 1F-B1',
   '9-1번 출구(교통섬) 측 , B1-1F'],
  '운행방향': ['상행', '하행', '상행', '상행', '하행', '하행', '상행', '하행', '상행']},
 '시청': {'호선': '1',
  '승강기번호': ['1808-119',
   '1808-120',
   '1809-239',
   '1808-578',
   '1808-577',
   '1808-576',
   '1808-575'],
  '설치위치': ['5번 출구 측, 1F-B1',
   '5번 출구 측, B1-1F',
   '1번 출구 측, B1-1F',
   '대합실(1호선 환승통로 방면), B1-B2',
   '대합실(1호선 환승통로 방면), B2-B1',
   '대합실(1호선 환승통로 방면), B1-B2',
   '대합실(1호선 환승통로 방면), B2-B1'],
  '운행방향': ['하행', '상행', '상행', '하행', '상행', '하행', '상행']},
 '종각': {'호선': '1',
  '승강기번호': ['1810-730', '1810-729'],
  '설치위치': ['1번 출구 측, 1F-B1', '1번 출구 측, B1-1F'],
  '운행방향': ['하행', '상행']},
 '종로가': {'호선': '1',
  '승강기번호': ['180

#### 에스컬레이터 완료

### 휠체어경사로 시작

In [60]:
WC_1to8_Line_Df = pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/서울시 지하철 역사 노약자 장애인 편의시설 현황/(최종 전처리) 서울교통공사_휠체어리프트 설치현황_20230420.csv',encoding='cp949')
WC_1to8_Line_Df.head(20)

,호선,역명,호기,승강기번호,설치위치,운행구간
0,1,서울역,내부#1,1902-362,"C계단 측, B2-B1",B2-B1
1,1,신설동,외부#2,1903-467,"6번 출구 측, 1F-B1",1F-B1
2,1,신설동,내부#1,1903-415,"제기동 방면, B2-B1",B2-B1
3,1,신설동,내부#2,1903-417,"동묘앞 방면, B2-B1",B2-B1
4,1,신설동,내부#3,1903-416,"연결통로(대합실), B1-B1",B1-B1
5,1,신설동,내부#4,1903-414,"연결통로(대합실), B1-B1",B1-B1
6,1,청량리,내부#2,1901-053,"회기 방면8-2, B2-B1",B2-B1
7,2,한양대,내부#1,1900-169,"뚝섬 방면8-2, 1F-2F",1F-2F
8,2,용답,내부#2,1900-168,"신답 방면1-4, 1F-2F",1F-2F
9,2,용답,내부#1,1900-276,"성수 방면2-2, 1F-2F",1F-2F


In [61]:
# Station_Facility 변수 초기화
Station_Facility = {}

# 데이터프레임 정보를 가공하여 Station_Facility 변수에 저장
for _, row in WC_1to8_Line_Df.iterrows():
    station = row['역명']
    if station not in Station_Facility:
        Station_Facility[station] = {
            '호선': row['호선'],
            '승강기번호': [],
            '설치위치': []
    
        }
    Station_Facility[station]['승강기번호'].append(row['승강기번호'])
    Station_Facility[station]['설치위치'].append(row['설치위치'])
    
    

    
dict['Wheel_Chair_Lift'] = Station_Facility
dict['Wheel_Chair_Lift']

{'서울역': {'호선': 1, '승강기번호': ['1902-362'], '설치위치': ['C계단 측, B2-B1']},
 '신설동': {'호선': 1,
  '승강기번호': ['1903-467',
   '1903-415',
   '1903-417',
   '1903-416',
   '1903-414',
   '1903-465',
   '1903-461',
   '1903-462',
   '1903-418',
   '1903-463',
   '1903-464'],
  '설치위치': ['6번 출구 측, 1F-B1',
   '제기동 방면, B2-B1',
   '동묘앞 방면, B2-B1',
   '연결통로(대합실), B1-B1',
   '연결통로(대합실), B1-B1',
   '10번 출구 측, B1-1F',
   '신설동 방면1-1, B2-B1',
   '용두 방면1-1, B2-B1',
   '용두 방면4-4, B2-B1',
   '환승통로(1,2호선), B1-B1',
   '환승통로(1,2호선), B1-B1']},
 '청량리': {'호선': 1, '승강기번호': ['1901-053'], '설치위치': ['회기 방면8-2, B2-B1']},
 '한양대': {'호선': 2, '승강기번호': ['1900-169'], '설치위치': ['뚝섬 방면8-2, 1F-2F']},
 '용답': {'호선': 2,
  '승강기번호': ['1900-168', '1900-276'],
  '설치위치': ['신답 방면1-4, 1F-2F', '성수 방면2-2, 1F-2F']},
 '잠실': {'호선': 2,
  '승강기번호': ['1901-593', '1900789', '1900790', '1900791'],
  '설치위치': ['잠실쇼핑센터 측 연결계단, B1-B1',
   '몽촌토성 방면6-4 환승통로 계단, B2-B1',
   '석촌 방면1-1 환승통로 계단, B3-B2',
   '사당 방면환승통로 끝 계단, 통로-B2']},
 '삼성': {'호선': 2, '승강기번호': ['1901-4

#### 휠체어리프트 완료

### 수유실 시작

In [65]:
Feeding_Room_Df =  pd.read_csv('C:/Users/dlwhd/Downloads/2023 Hackton/Info_Data/(최종 전처리) 엘베,휠체어리프트,수유 여부 - 서울시 지하철 역사 편의시설 현황.csv', encoding='cp949')
Feeding_Room_Df

,지하철역ID,역명,호선,엘리베이터유무,휘체어리프트유무,환승주차자유무,자전거보관소유무,무인민원발급기유무,환전키오스크유무,기차예매역유무,문화공간유무,문화공간유무.1,유아수유방유무
0,150,서울역,1,Y,Y,N,NaN,N,N,N,N,Y,False
1,151,시청,1,Y,N,N,NaN,N,N,N,N,Y,False
2,152,종각,1,Y,N,N,NaN,Y,N,N,N,N,False
3,153,종로3가,1,Y,N,N,NaN,N,N,N,Y,N,True
4,154,종로5가,1,Y,N,N,NaN,N,N,N,N,N,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,4134,송파나루,9,Y,N,N,NaN,Y,N,N,N,N,True
286,4135,한성백제,9,Y,N,N,NaN,N,N,N,N,N,True
287,4136,올림픽공원,9,Y,N,N,NaN,N,N,N,N,N,True
288,4137,둔촌오륜,9,Y,N,N,NaN,N,N,N,N,N,True


In [66]:
# Station_Facility 변수 초기화
Station_Facility = {}

# 데이터프레임 정보를 가공하여 Station_Facility 변수에 저장
for _, row in Feeding_Room_Df.iterrows():
    station = row['역명']
    if station not in Station_Facility:
        Station_Facility[station] = {
            '호선': row['호선'],
            '유아수유방유무': []
    
        }
    Station_Facility[station]['유아수유방유무'].append(row['유아수유방유무'])
    
    
    

    
dict['Feeding_Room_Df'] = Station_Facility
dict['Feeding_Room_Df']

{'서울역': {'호선': 1, '유아수유방유무': [False, True]},
 '시청': {'호선': 1, '유아수유방유무': [False, False]},
 '종각': {'호선': 1, '유아수유방유무': [False]},
 '종로3가': {'호선': 1, '유아수유방유무': [True, True, False]},
 '종로5가': {'호선': 1, '유아수유방유무': [False]},
 '동대문': {'호선': 1, '유아수유방유무': [True, False]},
 '신설동': {'호선': 1, '유아수유방유무': [False, True]},
 '제기동': {'호선': 1, '유아수유방유무': [False]},
 '청량리': {'호선': 1, '유아수유방유무': [False]},
 '동묘앞': {'호선': 1, '유아수유방유무': [False, True]},
 '까치산': {'호선': 2, '유아수유방유무': [True, True]},
 '을지로입구': {'호선': 2, '유아수유방유무': [False]},
 '을지로3가': {'호선': 2, '유아수유방유무': [False, False]},
 '을지로4가': {'호선': 2, '유아수유방유무': [False, False]},
 '동대문역사문화공원': {'호선': 2, '유아수유방유무': [False, True, True]},
 '신당': {'호선': 2, '유아수유방유무': [False, False]},
 '상왕십리': {'호선': 2, '유아수유방유무': [False]},
 '왕십리': {'호선': 2, '유아수유방유무': [False, True]},
 '한양대': {'호선': 2, '유아수유방유무': [False]},
 '뚝섬': {'호선': 2, '유아수유방유무': [False]},
 '성수': {'호선': 2, '유아수유방유무': [True]},
 '건대입구': {'호선': 2, '유아수유방유무': [False, False]},
 '구의': {'호선': 2, '유아수유방유무': [False]},


In [49]:
# 'Facility' 컬럼 추가
Station_Info['Facility'] = ''

# 조건에 맞는 값을 할당
for index, row in Station_Info.iterrows():
    station = row['STATION_NM']
    line_num = row['LINE']
    if station in Station_Facility and Station_Facility[station]['호선'] == line_num:
        Station_Info.at[index, 'Facility'] = Station_Facility[station]

Station_Info.head(30)

,STATION_CD,STATION_NM,LINE_NUM,Facility,LINE
0,0150,서울역,1호선,"{'호선': '1', '승강기번호': ['0074-205', '0104-656', ...",1
1,0151,시청,1호선,"{'호선': '1', '승강기번호': ['0002-888', '0002-887', ...",1
2,0152,종각,1호선,"{'호선': '1', '승강기번호': ['0000-506', '0000-507', ...",1
3,0153,종로3가,1호선,,1
4,0154,종로5가,1호선,,1
5,0155,동대문,1호선,"{'호선': '1', '승강기번호': ['0000-906', '0000-907', ...",1
6,0156,신설동,1호선,"{'호선': '1', '승강기번호': ['0083-762', '0083-764', ...",1
7,0157,제기동,1호선,"{'호선': '1', '승강기번호': ['0043-600', '0027-701', ...",1
8,0158,청량리,1호선,"{'호선': '1', '승강기번호': ['0027-702', '0158-313'],...",1
9,0159,동묘앞,1호선,"{'호선': '1', '승강기번호': ['0000-946', '0000-943', ...",1


In [38]:
Station_Facility.keys()
type(Station_Facility['서울역']['호선'])

str

In [14]:
# Station_Facility = {}

# # 반복문을 통해 EV_Line_1_DF부터 EV_Line_8_DF까지 처리
# for i in range(1, 9):
#     df_name = f'EV_Line_{i}_Df'  # 데이터프레임 변수명 생성
#     df = locals()[df_name]  # 변수명을 통해 실제 데이터프레임 참조

#     # 중복된 값을 가지는 row들의 인덱스를 모아서 리스트에 저장
#     duplicated_indices = df[df.duplicated(['역명'], keep=False)].index.tolist()

#     # 인덱스에 해당하는 상세위치 값을 한 곳으로 모아 리스트 생성
#     for index in duplicated_indices:
#         row = df.loc[index]
#         station = row['역명']
#         location = row['상세위치']
#         if station in Station_Facility:
#             Station_Facility[station].append(location)
#         else:
#             Station_Facility[station] = [location]

# Station_Facility

### 번호가 아닌 문자열 호선

In [15]:
# # 중복된 값을 가지는 row들의 인덱스를 모아서 리스트에 저장
# duplicated_indices = EV_Line_Gyeongui_Jungang_Df[EV_Line_Gyeongui_Jungang_Df.duplicated(['역명'], keep=False)].index.tolist()

# # 인덱스에 해당하는 상세위치 값을 한 곳으로 모아 리스트 생성
# for index in duplicated_indices:
#     row = EV_Line_Gyeongui_Jungang_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]
        

# duplicated_indices = EV_Line_Gyeongchun_Df[EV_Line_Gyeongchun_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Gyeongchun_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]


# duplicated_indices = EV_Line_Suin_Bundang_Df[EV_Line_Suin_Bundang_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Suin_Bundang_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]
        
        
# duplicated_indices = EV_Line_New_Bundang_Df[EV_Line_New_Bundang_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_New_Bundang_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]

        
# duplicated_indices = EV_Line_Ever_Df[EV_Line_Ever_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Ever_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]
        
        
# duplicated_indices = EV_Line_Uijeongbu_Rail_Df[EV_Line_Uijeongbu_Rail_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Uijeongbu_Rail_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]
        
        
# duplicated_indices = EV_Line_Incheon1_Df[EV_Line_Incheon1_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Incheon1_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]

        
# duplicated_indices = EV_Line_Incheon2_Df[EV_Line_Incheon2_Df.duplicated(['역명'], keep=False)].index.tolist()

# for index in duplicated_indices:
#     row = EV_Line_Incheon2_Df.loc[index]
#     station = row['역명']
#     location = row['상세위치']
#     if station in Station_Facility:
#         Station_Facility[station].append(location)
#     else:
#         Station_Facility[station] = [location]        
        
        
# Station_Facility